In [ ]:
# | default_exp classes.DomoGroup


In [ ]:
# | exporti
from dataclasses import dataclass, field
from typing import List, Union

import httpx
import asyncio

from fastcore.basics import patch_to


import domolibrary.utils.chunk_execution as ce

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.utils.DictDot as util_dd

import domolibrary.routes.group as group_routes

# import domolibrary.routes.publish as publish_routes

import domolibrary.classes.DomoUser as dmu


In [ ]:
#| export 

from domolibrary.routes.group import GroupType_Enum, SearchGroups_Error


# Group Membership

In [ ]:
# | export
class UpdateGroupMembership(de.DomoError):
    def __init__(self, member_name, group_name, domo_instance):
        super().__init__(domo_instance=domo_instance,
                         message=f"unable to add {member_name} to {group_name}")


class GroupMembership:
    _add_member_ls: list[str]
    _remove_group_ls: list[str]

    _add_owners_ls: list[str]
    _remove_owner_ls: list[str]

    _current_member_ls: list[str]
    _current_owner_ls: list[str]

    group = None

    def __init__(self, group):
        self.group = group

        self._add_member_ls: [dict] = []
        self._remove_member_ls: [dict] = []

        self._add_owner_ls: [dict] = []
        self._remove_owner_ls: [dict] = []

        self._current_member_ls = []
        self._current_owner_ls = []

    def _add_to_list(self, member, list_to_update, debug_prn: bool = False):

        import domolibrary.classes.DomoUser as dmu

        match_obj = next((user_obj for user_obj in list_to_update if user_obj.get(
            'id') == member.id), None)
        if match_obj:
            print(f"➡️ {member}  of type {type(member).__name__} already in ls")
            return list_to_update

        if debug_prn:
            print(
                f"➡️ adding {member.id}  of type {type(member).__name__} to {self.group.name}")

        if isinstance(member, dmu.DomoUser):
            list_to_update.append({'id': str(member.id), 'type': 'USER'})

            return list_to_update

        if isinstance(member, DomoGroup):
            list_to_update.append({'id': str(member.id), 'type': 'GROUP'})

            return list_to_update

        member_name = getattr(member, 'name', None) or getattr(
            member, 'display_name', None) or "name not provided"

        raise UpdateGroupMembership(domo_instance=self.group.auth.domo_instance,
                                    group_name=self.group.name,
                                    member_name=member_name)

    def _add_member(self, member, debug_prn: bool = False):
        return self._add_to_list(member, self._add_member_ls, debug_prn)

    def _remove_member(self, member, debug_prn: bool = False):
        if type(member).__name__ == 'DomoGroup' and member.type == 'system':
            if debug_prn:
                print(f"remove_owner - skipping {member.name} type is {member.type}")
            return
        return self._add_to_list(member, self._remove_member_ls, debug_prn)

    def _add_owner(self, member, debug_prn: bool = False):
        return self._add_to_list(member, self._add_owner_ls, debug_prn)

    def _remove_owner(self, member, debug_prn: bool = False):
        if type(member).__name__ == 'DomoGroup' and member.type == 'system':
            if debug_prn:
                print(f"remove_owner - skipping {member.name} type is {member.type}")
            return
            
        return self._add_to_list(member, self._remove_owner_ls, debug_prn)

    def _reset_obj(self):
        self._add_member_ls = []
        self._remove_member_ls = []

        self._add_owner_ls = []
        self._remove_owner_ls = []

    async def _update_group_access(self,
                                   debug_api: bool = False, session: httpx.AsyncClient = None,
                                   ):

        res = await group_routes.update_group_membership(auth=self.group.auth,
                                                         group_id=self.group.id,
                                                         add_member_arr=self._add_member_ls,
                                                         remove_member_arr=self._remove_member_ls,
                                                         add_owner_arr=self._add_owner_ls,
                                                         remove_owner_arr=self._remove_owner_ls,
                                                         debug_api=debug_api,
                                                         session=session)
        self._reset_obj()

        # add
        # remove
        # set


## implement GET methods

In [ ]:
#| exporti
@patch_to(GroupMembership)
async def get_owners(
    self: GroupMembership,
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    import domolibrary.classes.DomoUser as dmu

    auth = auth or self.group.auth

    self._current_owner_ls = []

    res = await group_routes.get_group_owners(group_id=self.group.id, auth=self.group.auth)
    if return_raw:
        return res
    
    group_ids = [obj.get('id') for obj in res.response if obj.get('type') == 'GROUP']
    if group_ids:
        domo_groups = await ce.gather_with_concurrency(n = 60, * [DomoGroup.get_by_id(group_id=group_id, auth=auth) for group_id in group_ids])
        self._current_owner_ls += domo_groups
    
    user_ids = [obj.get('id') for obj in res.response if obj.get('type') == 'USER']
    if user_ids:
        domo_users = await dmu.DomoUsers.by_id(user_ids=user_ids, auth=auth, only_allow_one = False)
        self._current_owner_ls += domo_users
        
    self.group.owner_id_ls = group_ids + user_ids
    self.group.owner_ls = self._current_owner_ls 

    return self._current_owner_ls
    # return domo_users


In [ ]:
#| exporti
@patch_to(GroupMembership)
async def get_members(
    self: GroupMembership,
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    import domolibrary.classes.DomoUser as dmu

    auth = auth or self.group.auth

    self._current_member_ls = []

    res = await group_routes.get_group_membership(group_id=self.group.id, auth=self.group.auth)
    if return_raw:
        return res
    
    user_ids = [obj.get('userId') for obj in res.response ]
    if user_ids:
        domo_users = await dmu.DomoUsers.by_id(user_ids=user_ids, auth=auth, only_allow_one = False)
        self._current_member_ls += domo_users

    self.group.members_id_ls = user_ids
    self.group.members_ls = self._current_member_ls

    return self.group.members_ls


## implement CRUD methods

In [ ]:
# | exporti
@patch_to(GroupMembership)
async def add_members(
    self: GroupMembership,
    add_user_ls: list[dmu.DomoUser],
    return_raw: bool = False,
    debug_api: bool = False, 
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    self._reset_obj()

    [self._add_member(domo_user, debug_prn) for domo_user in add_user_ls]

    res = await self._update_group_access(debug_api=debug_api, session=session)

    if return_raw:
        return res

    return await self.get_members()


@patch_to(GroupMembership)
async def remove_members(
    self: GroupMembership,
    remove_user_ls: list[dmu.DomoUser],
    return_raw: bool = False,
    debug_api: bool = False, debug_prn: bool = False, session: httpx.AsyncClient = None,
):
    self._reset_obj()

    [self._remove_member(domo_user, debug_prn) for domo_user in remove_user_ls]

    res = await self._update_group_access(debug_api=debug_api, session=session)

    if return_raw:
        return res

    return await self.get_members()


@patch_to(GroupMembership)
async def set_members(
    self: GroupMembership,
    user_ls: list[dmu.DomoUser],
    return_raw: bool = False,
    debug_api: bool = False, debug_prn:bool = False ,session: httpx.AsyncClient = None,
):
    self._reset_obj()

    domo_users = await self.get_members()

    if debug_prn:
        print({'domo_users': domo_users, 'user_ls': user_ls})

    [self._add_member(domo_user, debug_prn) for domo_user in user_ls]
    [self._remove_member(domo_user, debug_prn) for domo_user in domo_users if domo_user not in user_ls]


    res= await self._update_group_access(debug_api=debug_api, session=session)

    if return_raw:
        return res

    return await self.get_members()


In [ ]:
# | exporti
@patch_to(GroupMembership)
async def add_owners(
    self: GroupMembership,
    add_owner_ls,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    self._reset_obj()

    [self._add_owner(domo_user, debug_prn) for domo_user in add_owner_ls]

    res = await self._update_group_access(debug_api=debug_api, session=session)

    if return_raw:
        return res

    return await self.get_owners()


@patch_to(GroupMembership)
async def remove_owners(
    self: GroupMembership,
    remove_owner_ls,
    return_raw: bool = False,
    debug_api: bool = False, debug_prn: bool = False, session: httpx.AsyncClient = None,
):
    self._reset_obj()

    [self._remove_owner(domo_user, debug_prn) for domo_user in remove_owner_ls]

    res = await self._update_group_access(debug_api=debug_api, session=session)

    if return_raw:
        return res

    return await self.get_owners()


@patch_to(GroupMembership)
async def set_owners(
    self: GroupMembership,
    owner_ls,
    return_raw: bool = False,
    debug_api: bool = False, debug_prn: bool = False, session: httpx.AsyncClient = None,
):
    self._reset_obj()

    domo_users = await self.get_owners()

    if debug_prn:
        print({'domo_users': domo_users, 'user_ls': owner_ls})

    [self._add_owner(domo_user, debug_prn) for domo_user in owner_ls]
    [self._remove_owner(domo_user, debug_prn)
     for domo_user in domo_users if domo_user not in owner_ls]

    res = await self._update_group_access(debug_api=debug_api, session=session)

    if return_raw:
        return res

    return await self.get_owners()


# Domo Group

In [ ]:
# | export
@dataclass
class DomoGroup:
    auth: dmda.DomoAuth = field(repr=False, default=None)
    id: str = None
    name: str = None
    type: str = None
    description: str = None
    members_id_ls: list[str] = field(repr=False, default_factory=list)
    owner_id_ls: list[str] = field(repr=False, default_factory=list)
    
    members_ls: list[dict] = field(repr=False, default_factory=list)
    owner_ls : list[dict]= field(repr = False, default_factory = list)

    def __post_init__(self):
        # self.domo_instance = self.domo_instance or auth.domo_instance
        self.Membership = GroupMembership(self)

    @classmethod
    def _from_group_json(cls, auth: dmda.DomoAuth, json_obj):
        dd = json_obj

        if not isinstance(json_obj, util_dd.DictDot):
            dd = util_dd.DictDot(json_obj)

        return cls(
            auth=auth,
            id=dd.id or dd.groupId,
            name=dd.name,
            description=dd.description,
            type=dd.type or dd.groupType,
            members_id_ls=dd.userIds,
            owner_ls=dd.owners,
        )

    @classmethod
    def _from_grouplist_json(cls, auth: dmda.DomoAuth, json_obj):
        dd = json_obj

        if not isinstance(json_obj, util_dd.DictDot):
            dd = util_dd.DictDot(json_obj)

        return cls(
            auth=auth,
            id=dd.groupId,
            name=dd.name,
            description=dd.description,
            type=dd.groupType,
            owner_ls = dd.owners,
            owner_id_ls = [owner.id for owner in dd.owners],
            
            members_ls=dd.groupMembers,
            members_id_ls= [member.id for member in dd.members]
        )


    @staticmethod
    def _groups_to_domo_group(json_list, auth: dmda.DomoAuth) -> List[dict]:
        domo_groups = [
            DomoGroup._from_group_json(auth=auth, json_obj=json_obj)
            for json_obj in json_list
        ]

        return domo_groups

## GET Methods

In [ ]:
# | exporti
@patch_to(DomoGroup, cls_method=True)
async def get_by_id(
    cls,
    auth: dmda.DomoAuth,
    group_id: str,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.get_group_by_id(
        auth=auth, group_id=group_id, debug_api=debug_api, session=session
    )
    if return_raw:
        return res

    if res.status != 200:
        raise Exception()

    dg = cls._from_group_json(auth=auth, json_obj=res.response)

    # await dg.Membership.get_owners()
    # await dg.Membership.get_members() # disabled because causes recursion
    return dg


#### sample get_by_id


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

res = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id, return_raw= False)

res

DomoGroup(id=2146122228, name='test', type='closed', description='with_description')

In [ ]:
# | exporti
@patch_to(DomoGroup, cls_method=True)
async def search_by_name(
    cls,
    auth: dmda.DomoAuth,
    group_name: str,
    is_exact_match: bool = True,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    
    res = await group_routes.search_groups_by_name(
        auth=auth,
        search_name=group_name,
        debug_api=debug_api,
        is_exact_match=is_exact_match,
        session=session,
    )

    if return_raw:
        return res

    if isinstance(res.response, list):
        return cls._groups_to_domo_group(res.response, auth)

    return cls._from_group_json(auth=auth, json_obj=res.response)

#### sample search_by_name


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group 2q32"

try:
    res = await DomoGroup.search_by_name(group_name=group_name, auth=token_auth )
    print(res)

except SearchGroups_Error as e:
    print(e)


search_groups_by_name: Status 200 - There is no exact match for Test Group 2q32 at domo-community


# Sample Group Membership

## GET Methods
#### sample get_owners

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group = DomoGroup(id=1781661643, auth=token_auth)

await group.Membership.get_owners()

print(group.owner_id_ls) #does not differentiate between users and groups

group.owner_ls

['1781661643', '822382906', '1893952720']


[DomoGroup(id=1781661643, name='TestMembership', type='open', description='group with membership'),
 DomoGroup(id=822382906, name='Grant: Manage all groups', type='system', description='Create, edit and delete any groups. Add and remove people from any group.'),
 DomoUser(id='1893952720', title=None, department=None, display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=810756122, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)]

#### sample get_members

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group = DomoGroup(id=1781661643, auth=token_auth)

await group.Membership.get_members()

[DomoUser(id='696468809', title=None, department=None, display_name='Abish Srinath', email_address='abish.srinath@domo.com', role_id=2097317660, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
 DomoUser(id='1345737456', title=None, department=None, display_name='Aditya Jain', email_address='jain_aditya95@yahoo.com', role_id=2, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
 DomoUser(id='1141078945', title=None, department=None, display_name='Justin Pumford', email_address='jpumford@gmail.com', role_id=2097317660, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, emplo

## CRUD Methods
#### sample add_members

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old membership {', '.join([str(id) for id in domo_group.members_id_ls])}")


new_membership = await dmu.DomoUsers.by_id(user_ids=[1141078945, 1833256765], auth=token_auth, only_allow_one=False)
await domo_group.Membership.add_members(add_user_ls=new_membership, debug_prn=False)

for user in new_membership :
    if user in domo_group.members_ls:
        print(f"user {user.id} - {user.display_name} successfully added")

validate old membership 1141078945, 1725780094
user 1833256765 - Aaron Schofield successfully added
user 1141078945 - Justin Pumford successfully added


#### sample remove_members

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old membership {', '.join([str(id) for id in domo_group.members_id_ls])}")


remove_membership = await dmu.DomoUsers.by_id(user_ids=[1542225148], auth=token_auth, only_allow_one=False)
await domo_group.Membership.remove_members(remove_user_ls=remove_membership, debug_prn=False)

for user in remove_membership:
    if user  not in domo_group.members_ls:
        print(f"user {user.id} - {user.display_name} successfully removed")

validate old membership 1141078945, 1725780094, 1833256765
user 1542225148 - Adam Landefeld successfully removed


#### sample set_members

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old membership {', '.join([str(id) for id in domo_group.members_id_ls])}")


set_membership = await dmu.DomoUsers.by_id(user_ids=[1141078945, 1542225148, 1725780094,
                                                        ], auth=token_auth, only_allow_one=False)
await domo_group.Membership.set_members(user_ls=set_membership, debug_prn=False)

test_members = set_membership
for member in domo_group.members_ls:
    if member not in test_members:
        test_members.append(member)
                                                     
for user in test_members:
    if user not in set_membership:
        print(f"user {user.id} - {user.display_name} does not belong")

    if user in set_membership:
        print(f"user {user.id} - {user.display_name} belongs")

validate old membership 1141078945, 1725780094, 1833256765
user 1542225148 - Adam Landefeld belongs
user 1725780094 - Brett Gibson belongs
user 1141078945 - Justin Pumford belongs


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

membership = [dmu.DomoUser(id=1141078945, auth=token_auth),
              dmu.DomoUser(id = 1725780094, auth = token_auth)]


domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate membership {', '.join([str(id) for id in domo_group.members_id_ls])}")

# await domo_group.Membership.get_members()

await domo_group.Membership.set_members(user_ls=membership , debug_prn = False)


validate membership 1141078945, 1542225148, 1725780094


[DomoUser(id='1725780094', title=None, department=None, display_name='Brett Gibson', email_address='Brett.Gibson@domo.com', role_id=2, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
 DomoUser(id='1141078945', title=None, department=None, display_name='Justin Pumford', email_address='jpumford@gmail.com', role_id=2097317660, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)]

#### sample add_owners

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)
await domo_group.Membership.get_owners()

print(
    f"validate old ownership {', '.join([str(id) for id in domo_group.owner_id_ls])}")


new_ownership_user = await dmu.DomoUsers.by_id(user_ids=[1345408759], auth=token_auth, only_allow_one=False)
new_ownership_group = await DomoGroup.get_by_id(group_id=1028365465, auth = token_auth)
new_ownership = new_ownership_user + [new_ownership_group]

await domo_group.Membership.add_owners(add_owner_ls=new_ownership, debug_prn=False)

for user in new_ownership:
    if user in domo_group.owner_ls:
        if isinstance(user, dmu.DomoUser) :
            print(f"user {user.id} - {user.display_name} successfully added")
        if isinstance(user, DomoGroup):
            print(f"user {user.id} - {user.name} successfully added")


validate old ownership 1028365465, 822382906, 797235532
user 1345408759 - Alexis Lorenz (DataMaven) successfully added
user 1028365465 - Team Alpha successfully added


#### sample remove_owners

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old ownership {', '.join([str(id) for id in domo_group.owner_id_ls])}")

remove_owner = await dmu.DomoUsers.by_id(user_ids=[797235532], auth=token_auth, only_allow_one=False)

await domo_group.Membership.remove_owners(remove_owner_ls=remove_owner, debug_prn=False)


for user in remove_owner:
    if user not in domo_group.owner_ls:
        print(f"user {user.id} - {user.display_name} successfully removed")


validate old ownership 
user 797235532 - Teddy Murphy successfully removed


#### set_ownership

In [ ]:

import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


group_id = 2146122228
domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate old ownership {', '.join([str(id) for id in domo_group.members_id_ls])}")


set_ownership_user = await dmu.DomoUsers.by_id(user_ids=[797235532], auth=token_auth, only_allow_one=False)
set_ownership_group = await ce.gather_with_concurrency(n = 60, *[DomoGroup.get_by_id(group_id=group_id, auth=token_auth) for group_id in [1028365465,822382906]])
set_ownership = set_ownership_user + set_ownership_group

await domo_group.Membership.set_owners(owner_ls=set_ownership, debug_prn=False)

test_owners = set_ownership
for owner in domo_group.owner_ls:
    if owner not in test_owners:
        test_owners.append(owner)
                                                     
for owner in test_owners:
    if owner not in set_ownership:
        print(
            f"{type(owner).__name__} {owner.id} - {getattr(owner, 'display_name', None) or getattr(owner, 'name', None)} does not belong")

    if user in set_ownership:
        print(
            f"{type(owner).__name__} {owner.id} - {getattr(owner, 'display_name', None) or getattr(owner, 'name', None)} belongs")


validate old ownership 1141078945, 1725780094
DomoUser 797235532 - Teddy Murphy belongs
DomoGroup 1028365465 - Team Alpha belongs
DomoGroup 822382906 - Grant: Manage all groups belongs


# Domo Groups

In [ ]:
# | export
class DomoGroups:
    def __init__(self):
        pass

    @staticmethod
    def _groups_to_domo_group(json_list, auth: dmda.DomoAuth):

        return [
            DomoGroup._from_group_json(auth=auth, json_obj=json_obj)
            for json_obj in json_list
        ]

### get_all_groups

In [ ]:
# | export
@patch_to(DomoGroups, cls_method=True)
async def get_all_groups(
    cls: DomoGroups,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.get_all_groups(
        auth=auth, debug_api=debug_api, session=session
    )

    if len(res.response) > 0:
        json_list = res.response

        return cls._groups_to_domo_group(json_list=json_list, auth=auth)

    else:
        return []


#### sample implementation of get_all_groups


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_groups = await DomoGroups.get_all_groups(auth=token_auth)



### Toggle Group Visibility

In [ ]:
# |exporti

@patch_to(DomoGroups, cls_method=True)
async def toggle_system_group_visibility(cls: DomoGroups,auth: dmda.DomoAuth,
                                         is_hide_system_groups: bool,
                                         debug_api: bool = False):

    return await group_routes.toggle_system_group_visibility(auth=auth,
                                                is_hide_system_groups=is_hide_system_groups,
                                                debug_api=debug_api)


#### sample toggle_system_group_visibility

In [ ]:
import os
import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoGroups.toggle_system_group_visibility(auth=token_auth, is_hide_system_groups = False)

all_groups_with_hidden = await DomoGroups.get_all_groups(auth = token_auth)

await DomoGroups.toggle_system_group_visibility(auth=token_auth, is_hide_system_groups = True)

all_groups = await DomoGroups.get_all_groups(auth = token_auth)


print(f"there are {len(all_groups)} standard groups, and {len(all_groups_with_hidden)} groups including system groups")




toggling group visiblity in domo-community show system groups


toggling group visiblity in domo-community hiding system groups
there are 34 standard groups, and 107 groups including system groups


# CRUD Group

In [ ]:
#| exporti

@patch_to(GroupMembership)
async def add_owner_manage_groups_role(self : GroupMembership):
    
    await DomoGroups.toggle_system_group_visibility(auth = self.group.auth, is_hide_system_groups=False)
    
    grant_group = await DomoGroup.search_by_name(auth =self.group.auth , group_name = 'Grant: Manage all groups')
       
    await self.add_owners(add_owner_ls = [grant_group])

    await DomoGroups.toggle_system_group_visibility(auth = self.group.auth, is_hide_system_groups=True)

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group ac"

domo_group = await DomoGroup.search_by_name(auth = token_auth, group_name=group_name)
await domo_group.Membership.add_owner_manage_groups_role()



toggling group visiblity in domo-community show system groups
toggling group visiblity in domo-community hiding system groups


In [ ]:
# | exporti

@patch_to(DomoGroup, cls_method=True)
async def create_from_name(
    cls: DomoGroup,
    auth: dmda.DomoAuth,
    group_name: str = None,
    group_type: str = "open",  # use GroupType_Enum
    description: str = None,
    is_include_manage_groups_role: bool = True,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.create_group(
        auth=auth,
        group_name=group_name,
        group_type=group_type,
        description=description,
        debug_api=debug_api,
    )

    
    domo_group = cls._from_group_json(auth=auth, json_obj=res.response)

    await domo_group.Membership.add_owner_manage_groups_role()

    return domo_group


#### sample create_from_name


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group ac"

try:
    res = await DomoGroup.create_from_name(group_name=group_name, auth=token_auth)
    print(res)
except group_routes.CreateGroup_Error as e:
    print(e)

create_group: Status 400 - Test Group ac already exists. Choose a different group_name at domo-community


## update metadata

In [ ]:
# | exporti

@patch_to(DomoGroup)
async def update_metadata(
    self: DomoGroup,
    auth: dmda.DomoAuth = None,

    group_name: str = None,
    group_type: str = None,  # use GroupType_Enum
    description: str = None,
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth
    
    res = await group_routes.update_group(
        auth=auth,
        group_id = self.id,
        group_name = group_name,
        group_type = group_type,
        description = description,
        debug_api = debug_api,
        session = session)

    if return_raw:
        return res
    
    updated_group = await DomoGroup.get_by_id(auth = auth, group_id = self.id )
    

    self.name = updated_group.name or self.name
    self.description = updated_group.description or self.description
    self.type = updated_group.type or self.type

    return self

#### sample update_metadata

In [ ]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 1259653287

description = f'update metadata - updated {dt.date.today()}'

domo_group = await DomoGroup.get_by_id(group_id = group_id, auth = token_auth)

await domo_group.update_metadata(description = description,  return_raw=False)

DomoGroup(id=1259653287, name='Test Group 2_deleted_7ece7d46-48ef-11ee-b1a1-12193c00dc43', type='open', description='update metadata - updated 2023-09-01')

In [ ]:
# | hide
import nbdev
nbdev.nbdev_export()


SyntaxError: positional argument follows keyword argument (<unknown>, line 34)

## Upsert Groups

In [ ]:
# | exporti
@patch_to(DomoGroup, cls_method=True)
async def upsert(
    cls: DomoGroup,
    auth: dmda.DomoAuth,
    group_name: str,
    group_type: str = None, # if create_group, use routes.class.GroupType_Enum
    description: str = None,
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):

    try:
        res = await group_routes.search_groups_by_name(
            auth=auth,
            search_name=group_name,
            debug_api=debug_api,
            is_exact_match=True,
            session=session,
        )

        domo_group = cls._from_group_json(auth=auth, json_obj=res.response)
        
        return await domo_group.update_metadata(
            group_type = group_type,
            description = description,
            debug_api = debug_api)

    except group_routes.SearchGroups_Error as e:
        
        return await DomoGroup.create_from_name(
                auth=auth, 
                group_name=group_name, 
                group_type = group_type,
                description = description,
                debug_api=debug_api, session=session)

        return e

#### sample upsert

In [ ]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group"
description = f"updated via upsert_group on {dt.date.today()}"

await DomoGroup.upsert(group_name=group_name, auth=token_auth , description = description)

DomoGroup(id=1513712315, name='Test Group', type='closed', description='updated via upsert_group on 2023-09-01')

In [ ]:
from nbev import nbdev_export
nbdev.nbdev_export()